In [76]:
# imports
import cv2
from tensorflow.python.lib.io import file_io
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.optimizers import Adam
from keras import backend as K
from keras.layers import Activation, Dropout, Flatten, Dense, Input, Conv2D, MaxPooling2D, BatchNormalization, Concatenate, ReLU, LeakyReLU
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

import tensorflow as tf


In [77]:
%matplotlib inline

In [78]:
# constants
TRAIN_PATH = '../stage1_data/train'
LABEL_PATH = '../stage1_labels/train.csv'
COLORS = ['red','green', 'blue', 'yellow']
IMAGE_SIZE = 512
BATCH_SIZE = 2
SHAPE = (192, 192, 4)
THRESHOLD = 0.05


name_label_dict = {
0:  'Nucleoplasm',
1:  'Nuclear membrane',
2:  'Nucleoli',   
3:  'Nucleoli fibrillar center',
4:  'Nuclear speckles',
5:  'Nuclear bodies',
6:  'Endoplasmic reticulum',   
7:  'Golgi apparatus',
8:  'Peroxisomes',
9:  'Endosomes',
10:  'Lysosomes',
11:  'Intermediate filaments',
12:  'Actin filaments',
13:  'Focal adhesion sites',   
14:  'Microtubules',
15:  'Microtubule ends',  
16:  'Cytokinetic bridge',   
17:  'Mitotic spindle',
18:  'Microtubule organizing center',  
19:  'Centrosome',
20:  'Lipid droplets',
21:  'Plasma membrane',   
22:  'Cell junctions', 
23:  'Mitochondria',
24:  'Aggresome',
25:  'Cytosol',
26:  'Cytoplasmic bodies',   
27:  'Rods & rings' }

In [79]:
# -----------------------------------------
# get a list of unique specimen ids
# -----------------------------------------
def get_specimen_ids(path):

    # get a list of all the images
    file_list = file_io.list_directory(path)
    
    # truncate the file names to make a specimen id
    specimen_ids = [f[:36] for f in file_list]
    
    # eliminate duplicates
    specimen_ids = list(set(specimen_ids))
    
    return specimen_ids

# unit test
specimen_list = get_specimen_ids(TRAIN_PATH)
print(specimen_list)

['000c99ba-bba4-11e8-b2b9-ac1f6b6435d0', '001bcdd2-bbb2-11e8-b2ba-ac1f6b6435d0', '001838f8-bbca-11e8-b2bc-ac1f6b6435d0', '0020af02-bbba-11e8-b2ba-ac1f6b6435d0', '002daad6-bbc9-11e8-b2bc-ac1f6b6435d0']


In [80]:
def get_image_fname(specimen_id, color, lo_res=True):

    # construct filename
    if lo_res:
        fname = TRAIN_PATH + '/' + specimen_id + '_' + color + '.png'
    else:
        fname = TRAIN_PATH + '/' + specimen_id + '_' + color + '.tif'
        
    return fname

# unit test
s = '000c99ba-bba4-11e8-b2b9-ac1f6b6435d0'
f = get_image_fname(s, 'red')
print(f)

../stage1_data/train/000c99ba-bba4-11e8-b2b9-ac1f6b6435d0_red.png


In [83]:
train_labels = pd.read_csv(LABEL_PATH)
train_labels.head(5)


,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [84]:
selection_list = ['000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0', s, '001838f8-bbca-11e8-b2bc-ac1f6b6435d0']
subset = train_labels.loc[train_labels['Id'].isin(selection_list)]
subset.head()
split_labels = (subset.loc[subset['Id'] == '000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0'])['Target'].str.split(' ')
print (split_labels)

1    [7, 1, 2, 0]
Name: Target, dtype: object


In [85]:
# ---------------------------------------
# Keras style run time data generator
# ---------------------------------------
class HproteinDataGenerator(keras.utils.Sequence):
    
    # ---------------------------------------------
    # Required function to initialize the class
    # ---------------------------------------------
    def __init__(self, 
                 specimen_ids, 
                 labels, 
                 batch_size=BATCH_SIZE, 
                 shape=SHAPE, 
                 shuffle=False, 
                 use_cache=False, 
                 augment=False):
       
        self.specimen_ids = specimen_ids       # list of features
        self.labels = labels                   # list of labels
        self.batch_size = batch_size           # batch size
        self.shape = shape                     # shape of features
        self.shuffle = shuffle                 # boolean for shuffle
        self.use_cache = use_cache             # boolean for use of cache
        self.augment = augment                 # boolean for image augmentation
        
    # -------------------------------------------------------
    # Required function to determine the number of batches
    # -------------------------------------------------------
    def __len__(self):
        return int(np.ceil(len(self.specimen_ids) / float(self.batch_size)))
    
    # -------------------------------------------------------
    # Required function to get a batch
    # -------------------------------------------------------
    def __getitem__(self, index):
        
        # get the list of specimen ids for this batch
        specimen_ids = self.specimen_ids[self.batch_size*index:self.batch_size*(index + 1)]
                
        # create a zeroed out numpy array to load the batch into
        feature_batch = np.zeros((len(specimen_ids), self.shape[0], self.shape[1], self.shape[2]))
        
        # load a batch of labels
        label_batch = self.labels[self.batch_size*index:self.batch_size*(index + 1)]
        print('label_batch shape -> {}'.format(label_batch[0].shape))
        
        # load a batch of images
        if self.use_cache:
            print("Error: use_cache not implemented!")
        else:
            for i, specimen_id in enumerate(specimen_ids):
                print(feature_batch[i].shape)
                feature_batch[i] = self.get_stacked_image(specimen_id)
            
        # augment images if desired
        if self.augment:
            print("Error: Image augmentation not implemented!")
            
        return feature_batch, label_batch
            
            
    # -----------------------------------------
    # get a single image
    # -----------------------------------------
    def get_single_image(self, specimen_id, color, lo_res=True):

        # get image file name
        fname = get_image_fname(specimen_id, color, lo_res)

        # read image as a 1-channel image
        image = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
        
        image = cv2.resize(image, (self.shape[0], self.shape[1]))

        return image
    
            
    # -----------------------------------------
    # get a stacked (4-channel) image
    # -----------------------------------------
    def get_stacked_image(self, specimen_id, lo_res=True):

        # create a numpy array to place the 1-channel images into
        image = np.zeros((self.shape[0], self.shape[1], 4), dtype=np.uint8)

        for n, color in enumerate(COLORS):

            # get a single image
            i = self.get_single_image(specimen_id, color, lo_res)

            # store it a channel
            image[:, :, n] = i

        return image


In [86]:
# -----------------------------------------------------------
# get the available specimen ids and corresponding labels
# -----------------------------------------------------------
def get_train_data(train_path, label_path):
    
    # get the list of specimen ids
    specimen_ids = get_specimen_ids(train_path)
    
    # get the labels for all specimen_ids
    label_data = pd.read_csv(label_path)
    
    # get the subset of labels that match the specimen images that are on TRAIN_PATH
    labels_subset = label_data.loc[label_data['Id'].isin(specimen_ids)]
    
    #
    # convert labels to trainer format
    #
    
    # set up the list that will contain the list of encoded labels for each specimen id
    labels = []
    
    # loop through each specimen_id
    for specimen_id in specimen_ids:
        
        # split the space separated multi-label into a list of individual labels
        split_labels = (labels_subset.loc[labels_subset['Id'] == specimen_id])['Target'].str.split(' ')

        # set up a numpy array to receive the encoded label
        l = np.zeros(28, dtype=np.uint8)

        # turn on the positive columns in the labels array
        for label in split_labels:
            l[np.uint8(label)] = 1
        
        labels.append(l)
        
    return np.array(specimen_ids), np.array(labels)

# unit test 
specimen_ids, labels = get_train_data(TRAIN_PATH, LABEL_PATH)
for sid, l in zip(specimen_ids, labels):
    print('Specimen Id: {} : Labels: {}'.format(sid, l))


Specimen Id: 000c99ba-bba4-11e8-b2b9-ac1f6b6435d0 : Labels: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Specimen Id: 001bcdd2-bbb2-11e8-b2ba-ac1f6b6435d0 : Labels: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Specimen Id: 001838f8-bbca-11e8-b2bc-ac1f6b6435d0 : Labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Specimen Id: 0020af02-bbba-11e8-b2ba-ac1f6b6435d0 : Labels: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
Specimen Id: 002daad6-bbc9-11e8-b2bc-ac1f6b6435d0 : Labels: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [87]:
# --------------------------------
# calculate the f1 statistic
# --------------------------------
def f1(y_true, y_pred):
    
    #y_pred = K.round(y_pred)
    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    
    return K.mean(f1)

# unit test
y_true = K.variable(np.ones(10, np.uint8))
y_pred = np.ones(10, np.uint8)
y_pred[0] = 0
y_pred = K.variable(y_pred)

ftest = f1(y_true, y_pred)
#ftest = tf.constant(5)

init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
print (sess.run(ftest))


0.9473683


In [88]:
# --------------------------------
# calculate the f1 loss
# --------------------------------

def f1_loss(y_true, y_pred):

    f = f1(y_true, y_pred)
    
    return 1 - K.mean(f)

# unit test
y_true = K.variable(np.ones(10, np.uint8))
y_pred = np.ones(10, np.uint8)
y_pred[0] = 0
y_pred = K.variable(y_pred)

ftest = f1_loss(y_true, y_pred)
#ftest = tf.constant(5)

init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
print (sess.run(ftest))

0.052631676


In [89]:
def create_model(input_shape):

    dropRate = 0.25
    
    init = Input(input_shape)
    x = BatchNormalization(axis=-1)(init)
    x = Conv2D(8, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(8, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(16, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    c1 = Conv2D(16, (3, 3), padding='same')(x)
    c1 = ReLU()(c1)
    c2 = Conv2D(16, (5, 5), padding='same')(x)
    c2 = ReLU()(c2)
    c3 = Conv2D(16, (7, 7), padding='same')(x)
    c3 = ReLU()(c3)
    c4 = Conv2D(16, (1, 1), padding='same')(x)
    c4 = ReLU()(c4)
    x = Concatenate()([c1, c2, c3, c4])
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    x = Conv2D(32, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    x = Conv2D(64, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    #x = Conv2D(256, (1, 1), activation='relu')(x)
    #x = BatchNormalization(axis=-1)(x)
    #x = MaxPooling2D(pool_size=(2, 2))(x)
    #x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(28)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(0.1)(x)
    x = Dense(28)(x)
    x = Activation('sigmoid')(x)
    
    model = Model(init, x)
    
    return model

# unit test
model = create_model(SHAPE)
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-03),
    metrics=['acc',f1])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 192, 192, 4)  0                                            
__________________________________________________________________________________________________
batch_normalization_37 (BatchNo (None, 192, 192, 4)  16          input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_41 (Conv2D)              (None, 190, 190, 8)  296         batch_normalization_37[0][0]     
__________________________________________________________________________________________________
re_lu_45 (ReLU)                 (None, 190, 190, 8)  0           conv2d_41[0][0]                  
__________________________________________________________________________________________________
batch_norm

In [94]:
# get data
specimen_ids, labels = get_train_data(TRAIN_PATH, LABEL_PATH)

train_set_sids = specimen_ids[0:3]
train_set_lbls = labels[0:3]

val_set_sids = specimen_ids[3:]
val_set_lbls = labels[3:]

# create data generators
tg = HproteinDataGenerator(train_set_sids, train_set_lbls)
vg = HproteinDataGenerator(val_set_sids, val_set_lbls)

(5,)
(5, 28)
(3,)
(3, 28)
(2,)
(2, 28)


In [91]:
checkpoint = ModelCheckpoint('./base.model', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)
reduceLROnPlato = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='min')

In [92]:
epochs = 1

use_multiprocessing = False # DO NOT COMBINE MULTIPROCESSING WITH CACHE! 
workers = 1 # DO NOT COMBINE MULTIPROCESSING WITH CACHE! 

hist = model.fit_generator(
    tg,
    steps_per_epoch=len(tg),
    validation_data=vg,
    validation_steps=8,
    epochs=epochs,
    use_multiprocessing=use_multiprocessing,
    workers=workers,
    verbose=1,
    callbacks=[checkpoint])

Epoch 1/1
label_batch shape -> (28,)
(192, 192, 4)
label_batch shape -> (28,)
(192, 192, 4)
label_batch shape -> (28,)
(192, 192, 4)
(192, 192, 4)
1/2 [==============>...............] - ETA: 6s - loss: 0.6931 - acc: 0.9643 - f1: 0.0357label_batch shape -> (28,)
(192, 192, 4)
(192, 192, 4)
label_batch shape -> (28,)
(192, 192, 4)
label_batch shape -> (28,)
(192, 192, 4)
label_batch shape -> (28,)
(192, 192, 4)
label_batch shape -> (28,)
(192, 192, 4)
label_batch shape -> (28,)
(192, 192, 4)
label_batch shape -> (28,)
(192, 192, 4)
label_batch shape -> (28,)
(192, 192, 4)
label_batch shape -> (28,)
(192, 192, 4)
label_batch shape -> (28,)
(192, 192, 4)
2/2 [==============================] - 9s 5s/step - loss: 0.7563 - acc: 0.6667 - f1: 0.0423 - val_loss: 6.1260 - val_acc: 0.4286 - val_f1: 0.0357

Epoch 00001: val_loss improved from inf to 6.12604, saving model to ./base.model
